# Models that we have used so far with 160 Data

In [1]:
import tensorflow as tf
import numpy as np
from keras.regularizers import l2

def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 11, 56, 1)
      yield [data.astype(np.float32)]

2023-10-29 15:37:38.786669: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Presence detection

In [12]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(11, 56, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(0.0001))
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
model.save('saved_model/presence_det')

saved_model_dir = 'saved_model/presence_det'

INFO:tensorflow:Assets written to: saved_model/presence_det/assets


INFO:tensorflow:Assets written to: saved_model/presence_det/assets


In [15]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

# Save the model.
with open('saved_model/presence_det.tflite', 'wb') as f:
  f.write(tflite_quant_model)

2023-10-29 15:30:26.167966: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-10-29 15:30:26.168000: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-10-29 15:30:26.168179: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: saved_model/presence_det
2023-10-29 15:30:26.170346: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-10-29 15:30:26.170367: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: saved_model/presence_det
2023-10-29 15:30:26.175728: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-10-29 15:30:26.210575: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: saved_model/presence_det
2023-10-29 15:30:26.224331: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: s

## Distance estimation

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(11, 56, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dropout(0.04),
  tf.keras.layers.Dense(1, activation='linear', kernel_regularizer=l2(0.0001))
])
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

2023-10-29 15:37:48.624427: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 15:37:49.268691: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 42169 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:31:00.0, compute capability: 8.6


In [4]:
model.save('saved_model/dist')

saved_model_dir = 'saved_model/dist'

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

# Save the model.
with open('saved_model/dist.tflite', 'wb') as f:
  f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: saved_model/dist/assets


INFO:tensorflow:Assets written to: saved_model/dist/assets
2023-10-29 15:37:51.237752: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-10-29 15:37:51.237784: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-10-29 15:37:51.238682: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: saved_model/dist
2023-10-29 15:37:51.241157: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-10-29 15:37:51.241186: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: saved_model/dist
2023-10-29 15:37:51.267110: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-10-29 15:37:51.269499: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-10-29 15:37:51.352367: I tensorflow/cc/saved_model/loader.cc:212] Running initializatio

# Presence Detection Car original (novelda model)

In [6]:
model = tf.keras.Sequential()

img_h = 53
img_w = 86

conv_block = 3
start_f = 14
conv_kernel = 5
MP_dim = 2
drop_out_rate = 0.2
num_classes = 2

input_shape = [img_h, img_w, 1]


model.add(tf.keras.layers.MaxPool2D(pool_size=(MP_dim, MP_dim), input_shape=input_shape))

for i in range(conv_block):
    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                        kernel_size=(conv_kernel, conv_kernel),
                                        strides=(1, 1),
                                        padding='same',
                                        input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                        kernel_size=(conv_kernel, conv_kernel),
                                        strides=(1, 1),
                                        padding='same',
                                        input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))


# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(drop_out_rate))
#model.add(Dense(30, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=num_classes, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics='accuracy')

In [ ]:
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, img_h, img_w, 1)
      yield [data.astype(np.float32)]

model.save('saved_model/pd_novelda')

saved_model_dir = 'saved_model/pd_novelda'

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

# Save the model.
with open('saved_model/pd_novelda.tflite', 'wb') as f:
  f.write(tflite_quant_model)

## Subject recognition car

In [ ]:
model = tf.keras.Sequential()

img_h = 53
img_w = 86

conv_block = 2
start_f = 14
conv_kernel = 7
MP_dim = 2
drop_out_rate = 0.2
num_classes = 3

input_shape = [img_h, img_w, 1]


model.add(tf.keras.layers.MaxPool2D(pool_size=(MP_dim, MP_dim), input_shape=input_shape))

for i in range(conv_block):
    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                        kernel_size=(conv_kernel, conv_kernel),
                                        strides=(1, 1),
                                        padding='same',
                                        input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                        kernel_size=(conv_kernel, conv_kernel),
                                        strides=(1, 1),
                                        padding='same',
                                        input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))


# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(drop_out_rate))
#model.add(Dense(30, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=num_classes, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics='accuracy')

In [ ]:
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, img_h, img_w, 1)
      yield [data.astype(np.float32)]

model.save('saved_model/people_count_novelda')

saved_model_dir = 'saved_model/people_count_novelda'

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

# Save the model.
with open('saved_model/people_count.tflite', 'wb') as f:
  f.write(tflite_quant_model)

# People counting car 

In [ ]:
model = tf.keras.Sequential()

img_h = 53
img_w = 86

conv_block = 3
start_f = 12
conv_kernel = 7
MP_dim = 2
drop_out_rate = 0.2
num_classes = 1

input_shape = [img_h, img_w, 1]


model.add(tf.keras.layers.MaxPool2D(pool_size=(MP_dim, MP_dim), input_shape=input_shape))

for i in range(conv_block):
    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                        kernel_size=(conv_kernel, conv_kernel),
                                        strides=(1, 1),
                                        padding='same',
                                        input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                        kernel_size=(conv_kernel, conv_kernel),
                                        strides=(1, 1),
                                        padding='same',
                                        input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))


# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(drop_out_rate))
#model.add(Dense(30, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(units=num_classes, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics='accuracy')

In [ ]:
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, img_h, img_w, 1)
      yield [data.astype(np.float32)]

model.save('saved_model/sr_novelda')

saved_model_dir = 'saved_model/sr_novelda'

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()

# Save the model.
with open('saved_model/sr_novelda.tflite', 'wb') as f:
  f.write(tflite_quant_model)